# Healing Hugs Exploratory Analysis
Summary: This analysis takes a look at several variables to see if they are correlated to a husband received a healing hug from his wife. 

What is a healing hug, you ask? Just a one can consciously feel a physiological reponse to stress (think about when you are super stressed and your body physically feels different), this husband reports that with some hugs with his wife, he can feel the physiological reponse of muscle relaxation and lowered stressed. He presumes that during these hugs, his body releases healing hormones (or at least receives some form of healing).  

As he presumes this is healthy, he would like to received as many of these hugs as possible. Who wouldn't?!

He discovered I was getting into data science and began recording some variables with the hope one day I would be able to provide insight. After nearly 2.5 years, that day has come. 

At his request, I pre-processed the data to some extent to remove some information. The end of that preprossessing is where this project starts: healing_hugs.csv.